In [1]:
# initialization code
import numpy as np
from keras.datasets import mnist
%matplotlib inline
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.linear_model import LogisticRegression
from scipy.ndimage.interpolation import shift

import warnings
warnings.filterwarnings('ignore')

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.reshape(x_train, (len(x_train), 28*28))
x_test = np.reshape(x_test, (len(x_test), 28*28))
maxval = x_train.max()
x_train = x_train/maxval
x_test = x_test/maxval
x_val = x_train[:10000]
y_val = y_train[:10000]
x_train = x_train[10000:]
y_train = y_train[10000:]
x_train.shape

(50000, 784)

In [3]:
clf = LogisticRegression().fit(x_train, y_train)
y_pred_val = clf.predict(x_val)
print(f"Validation error: {np.sum(y_pred_val != y_val) / y_pred_val.shape[0]}")
y_pred_test = clf.predict(x_test)
print(f"Validation error: {np.sum(y_pred_test != y_test) / y_pred_test.shape[0]}")

Validation error: 0.0753
Validation error: 0.0737


In [13]:
clf = LogisticRegression(C=0.4, multi_class='multinomial', penalty='l2', solver='sag').fit(x_train, y_train)
y_pred_val = clf.predict(x_val)
print(f"Validation error: {np.sum(y_pred_val != y_val) / y_pred_val.shape[0]}")
y_pred_test = clf.predict(x_test)
print(f"Validation error: {np.sum(y_pred_test != y_test) / y_pred_test.shape[0]}")

Validation error: 0.0747
Validation error: 0.0728


In [4]:
# data argumentation

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

X_train_augmented = [image for image in x_train]
y_train_augmented = [image for image in y_train]

for dx, dy in ((1,0), (-1,0), (0,1), (0,-1)):
     for image, label in zip(x_train, y_train):
             X_train_augmented.append(shift_image(image, dx, dy))
             y_train_augmented.append(label)


# Shuffle the dataset
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = np.array(X_train_augmented)[shuffle_idx]
y_train_augmented = np.array(y_train_augmented)[shuffle_idx]

In [11]:
clf = LogisticRegression(C=0.4, multi_class='multinomial', penalty='l2', solver='sag')
clf.fit(X_train_augmented, y_train_augmented)
y_pred = clf.predict(x_val)
print(f"Validation error: {np.sum(y_pred != y_val) / y_pred.shape[0]}")

Validation error: 0.0772


In [14]:
# test
y_pred = clf.predict(x_test)
print(f"Validation error: {np.sum(y_pred != y_test) / y_pred.shape[0]}")

Validation error: 0.0728
